In [1]:
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
import re


plt.rcParams["figure.figsize"] = (16,8)



In [2]:
filename = "../datasets/DataCollector01.csv"
# csv 파일 읽기
df = pd.read_csv(filename,parse_dates=[0], na_values=[' '])
# df의 타입 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17281 entries, 0 to 17280
Data columns (total 44 columns):
 #   Column                                                                                                  Non-Null Count  Dtype         
---  ------                                                                                                  --------------  -----         
 0   (PDH-CSV 4.0) (                                                                                         17281 non-null  datetime64[ns]
 1   \\WIN-3OPFVMF4N3A\Memory\% Committed Bytes In Use                                                       17281 non-null  float64       
 2   \\WIN-3OPFVMF4N3A\Memory\Available MBytes                                                               17281 non-null  int64         
 3   \\WIN-3OPFVMF4N3A\Memory\Cache Bytes                                                                    17281 non-null  int64         
 4   \\WIN-3OPFVMF4N3A\Network Interface(Intel[R] 8

In [3]:
# df에서 colums 추출하기
col = list(map(lambda x:x.replace('\\','/'), df.columns))




In [4]:
# 설비 명 찾기
machineName = re.match('^//([^/]*)/', col[2]).group(1)
machineName

'WIN-3OPFVMF4N3A'

In [5]:
# 첫번째 열 이름 바꾸기
col[0] = "Date"



In [6]:
# Column 내 장비명을 삭제하여 정규화하기
col2 = map(lambda x: re.sub('^//([^/]*)/', '', x), col)


In [7]:
# df의 Columns 이름 변경
df.columns = col2



In [8]:
df.index

RangeIndex(start=0, stop=17281, step=1)

In [9]:
df["MachineName"] = machineName 
df.set_index(["Date", "MachineName"], inplace=True)

ret_cond = []

In [10]:
x =     ['avg90',    0.05, 0.05]

df90 = df.where(df <= df.quantile(q=(1-x[1]))).where(df >= df.quantile(q=x[2])).unstack().resample('1h').mean().interpolate()
df90["Type"] = x[0]
df90.set_index("Type", append=True, inplace=True )
ret_cond.append(df90.unstack())

In [11]:
x =     ['avg90H10',    0.05, 0.85]

df90 = df.where(df <= df.quantile(q=(1-x[1]))).where(df >= df.quantile(q=x[2])).unstack().resample('1h').mean().interpolate()
df90["Name"]= x[0]
df90.set_index("Name", append=True, inplace=True )
ret_cond.append(df90.unstack())

In [12]:
ret = pd.concat(ret_cond)







In [13]:
ret.info

<bound method DataFrame.info of                     Memory/% Committed Bytes In Use             \
                                    WIN-3OPFVMF4N3A              
                                              avg90   avg90H10   
Date                                                             
2020-02-05 17:00:00                       17.319669        NaN   
2020-02-05 18:00:00                       17.315030        NaN   
2020-02-05 19:00:00                       17.321841        NaN   
2020-02-05 20:00:00                       17.320201        NaN   
2020-02-05 21:00:00                       17.320500        NaN   
2020-02-05 22:00:00                       17.321738        NaN   
2020-02-05 23:00:00                       17.325864        NaN   
2020-02-06 00:00:00                       17.326553        NaN   
2020-02-06 01:00:00                       17.324077        NaN   
2020-02-06 02:00:00                       17.324308        NaN   
2020-02-06 03:00:00                       17

In [14]:
ret.resample (rule='1d', level=0).mean()

Memory/% Committed Bytes In Use            Memory/Available MBytes  \
                           WIN-3OPFVMF4N3A                    WIN-3OPFVMF4N3A   
                                     avg90   avg90H10                   avg90   
Date                                                                            
2020-02-05                       17.320692  17.410420             6950.058151   
2020-02-06                       17.322070  17.411541             6949.262645   

                        Memory/Cache Bytes                \
                           WIN-3OPFVMF4N3A                 
               avg90H10              avg90      avg90H10   
Date                                                       
2020-02-05  6953.694864       9.009619e+07  9.131839e+07   
2020-02-06  6953.399263       9.069107e+07  9.139809e+07   

           Network Interface(6TO4 Adapter)/Bytes Total/sec           \
                                           WIN-3OPFVMF4N3A            
                                                     avg90 avg90H10   
Date                                                                  
2020-02-05                                             0.0      0.0   
2020-02-06                                             0.0      0.0   

           Network Interface(6TO4 Adapter)/Output Queue Length           ...  \
                                               WIN-3OPFVMF4N3A           ...   
                                                         avg90 avg90H10  ...   
Date                                                                     ...   
2020-02-05                                                0.0       0.0  ...   
2020-02-06                                                0.0       0.0  ...   

           Processor(0)/% User Time           Processor(1)/% User Time  \
                    WIN-3OPFVMF4N3A                    WIN-3OPFVMF4N3A   
                              avg90  avg90H10                    avg90   
Date                                                                     
2020-02-05                 0.018982  0.018982                 0.028840   
2020-02-06                 0.017106  0.017106                 0.025153   

                     Processor(2)/% User Time            \
                              WIN-3OPFVMF4N3A             
            avg90H10                    avg90  avg90H10   
Date                                                      
2020-02-05  0.028840                 0.059399  0.312804   
2020-02-06  0.025153                 0.062276  0.312829   

           Processor(3)/% User Time           Processor(_Total)/% User Time  \
                    WIN-3OPFVMF4N3A                         WIN-3OPFVMF4N3A   
                              avg90  avg90H10                         avg90   
Date                                                                          
2020-02-05                 0.027671  0.027671                      0.040261   
2020-02-06                 0.028620  0.028620                      0.040849   

                      
                      
            avg90H10  
Date                  
2020-02-05  0.158036  
2020-02-06  0.159097  

[2 rows x 86 columns]

In [15]:
ret

Memory/% Committed Bytes In Use             \
                                    WIN-3OPFVMF4N3A              
                                              avg90   avg90H10   
Date                                                             
2020-02-05 17:00:00                       17.319669        NaN   
2020-02-05 18:00:00                       17.315030        NaN   
2020-02-05 19:00:00                       17.321841        NaN   
2020-02-05 20:00:00                       17.320201        NaN   
2020-02-05 21:00:00                       17.320500        NaN   
2020-02-05 22:00:00                       17.321738        NaN   
2020-02-05 23:00:00                       17.325864        NaN   
2020-02-06 00:00:00                       17.326553        NaN   
2020-02-06 01:00:00                       17.324077        NaN   
2020-02-06 02:00:00                       17.324308        NaN   
2020-02-06 03:00:00                       17.317053        NaN   
2020-02-06 04:00:00                       17.322297        NaN   
2020-02-06 05:00:00                       17.320287        NaN   
2020-02-06 06:00:00                       17.323555        NaN   
2020-02-06 07:00:00                       17.322482        NaN   
2020-02-06 08:00:00                       17.319601        NaN   
2020-02-06 09:00:00                       17.327148        NaN   
2020-02-06 10:00:00                       17.328213        NaN   
2020-02-06 11:00:00                       17.328915        NaN   
2020-02-06 12:00:00                       17.323852        NaN   
2020-02-06 13:00:00                       17.326627        NaN   
2020-02-06 14:00:00                       17.319087        NaN   
2020-02-06 15:00:00                       17.315204        NaN   
2020-02-06 16:00:00                       17.307662        NaN   
2020-02-06 17:00:00                       17.320338        NaN   
2020-02-05 17:00:00                             NaN  17.408286   
2020-02-05 18:00:00                             NaN  17.410303   
2020-02-05 19:00:00                             NaN  17.408054   
2020-02-05 20:00:00                             NaN  17.411039   
2020-02-05 21:00:00                             NaN  17.409931   
2020-02-05 22:00:00                             NaN  17.410970   
2020-02-05 23:00:00                             NaN  17.414360   
2020-02-06 00:00:00                             NaN  17.412649   
2020-02-06 01:00:00                             NaN  17.414560   
2020-02-06 02:00:00                             NaN  17.414742   
2020-02-06 03:00:00                             NaN  17.411620   
2020-02-06 04:00:00                             NaN  17.413257   
2020-02-06 05:00:00                             NaN  17.413313   
2020-02-06 06:00:00                             NaN  17.412266   
2020-02-06 07:00:00                             NaN  17.413184   
2020-02-06 08:00:00                             NaN  17.410952   
2020-02-06 09:00:00                             NaN  17.415119   
2020-02-06 10:00:00                             NaN  17.416916   
2020-02-06 11:00:00                             NaN  17.414883   
2020-02-06 12:00:00                             NaN  17.411810   
2020-02-06 13:00:00                             NaN  17.412114   
2020-02-06 14:00:00                             NaN  17.410525   
2020-02-06 15:00:00                             NaN  17.407939   
2020-02-06 16:00:00                             NaN  17.402267   
2020-02-06 17:00:00                             NaN  17.399615   

                    Memory/Available MBytes              Memory/Cache Bytes  \
                            WIN-3OPFVMF4N3A                 WIN-3OPFVMF4N3A   
                                      avg90     avg90H10              avg90   
Date                                                                          
2020-02-05 17:00:00             6950.890951          NaN       8.981740e+07   
2020-02-05 18:00:00             6950.442509          NaN    

In [16]:

ret = ret.unstack().stack(level=0)
ret['Date'] = date
ret['Filename'] = filename
ret.set_index(["Date", 'Filename'], append=True, inplace=True )

AttributeError: 'Series' object has no attribute 'stack'